In [10]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib

## Connecting to the database

In [11]:
db_path = '../db_logging/simlog.db'
conn = sqlite3.connect(db_path)
c = conn.cursor()

## Getting Ride lifecycle

In [41]:
sim_id_col = "Requests.sim_id as Sim_ID"
ride_id_col = "Requests.ride_id as Ride_ID"
origin_node_col = "Requests.origin_node as Origin"
dest_node_col = "Requests.dest_node as Destination"
request_ts_col = "Requests.execute_ts as Request_TS"
schedule_ts_col = "Schedules.execute_ts as Schedule_TS"
pickup_ts_col = "Pickups.execute_ts as Pickup_TS"
dropoff_ts_col = "Dropoffs.execute_ts as Dropoff_TS"

inner_join_str = "(((Requests INNER JOIN Schedules ON Requests.ride_id = Schedules.ride_id)" + \
    "INNER JOIN Pickups ON Requests.ride_id = Pickups.ride_id)" + \
    "INNER JOIN Dropoffs ON Requests.ride_id = Dropoffs.ride_id)"

query_str = "SELECT " + ','.join([sim_id_col, ride_id_col, origin_node_col, dest_node_col,
                                  request_ts_col, schedule_ts_col, pickup_ts_col, dropoff_ts_col]) + \
            " FROM " + inner_join_str + ";"


ride_life_cycle = pd.read_sql_query(query_str, conn)
ride_life_cycle.head()

buses = pd.read_sql_query("SELECT * FROM NumBusesIncrease WHERE NumBusesIncrease.sim_id > 3", conn)




In [42]:
m = ride_life_cycle['Sim_ID'].max()
# filter to latest 
ride_life_cycle = ride_life_cycle[ride_life_cycle['Sim_ID'] == m]

In [43]:
buses

,id,ride_id,execute_ts,location,num_buses,bus_id,sim_id
0,1,a1e2a267-86ff-43db-8337-3e218c289f5d,6.394587,203,11,10,4
1,2,60333705-ff8d-4891-b22f-4fab6330e4f0,6.426547,44,12,11,4
2,3,e453aa7a-82e5-4f15-98b3-0d69dbfc955e,6.473831,7,13,12,4
3,4,3c066878-9b6b-46ad-89c8-8c960139f66b,6.509724,184,14,13,4
4,5,43c8fc07-138d-4b66-96a1-3501cfc9b32f,6.522297,228,15,14,4
5,6,d064c616-9542-44d4-b500-8cd81051c83c,6.559690,64,16,15,4
6,7,797e9b20-c889-4e01-95b8-8faa8a14abde,6.594617,66,17,16,4
7,8,78b0a795-d79c-42c1-a1de-fd058ec97795,6.617110,61,18,17,4
8,9,5b6018ba-9e1b-4723-96aa-f729c00d3262,6.650618,154,19,18,4
9,10,c6066047-b71f-4e5a-a643-0a1958876b2d,6.682977,165,20,19,4


## Checking order of execution of events

In [23]:
#Column Names
ride_life_cycle.columns

Index(['Sim_ID', 'Ride_ID', 'Origin', 'Destination', 'Request_TS',
       'Schedule_TS', 'Pickup_TS', 'Dropoff_TS'],
      dtype='object')

Are all scheduled time stamps at or after requests?

In [24]:
((ride_life_cycle['Schedule_TS'] - ride_life_cycle['Request_TS']) >= 0).all()

True

Are all pickup time stamps at or after schedules?

In [25]:
((ride_life_cycle['Pickup_TS'] - ride_life_cycle['Schedule_TS']) >= 0).all()

True

Are all dropoff time stamps after pickup_ts_col schedules?

In [27]:
((ride_life_cycle['Dropoff_TS'] - ride_life_cycle['Pickup_TS']) >= 0).all()

False

In [28]:
problematic_rows = ride_life_cycle[ride_life_cycle['Dropoff_TS'] - ride_life_cycle['Pickup_TS'] < 0]

In [35]:
len(problematic_rows)/len(ride_life_cycle)

0.004942339373970346

In [36]:
problematic_rows

,Sim_ID,Ride_ID,Origin,Destination,Request_TS,Schedule_TS,Pickup_TS,Dropoff_TS
1464,4,dfa39b3d-9955-429c-a8e5-8384b6fdb225,7,223,6.799523,6.799523,10.092021,9.199493
1498,4,beea018d-9196-4e03-a88a-ddf451af72c7,59,7,6.805370,6.805370,9.199493,8.362521
1506,4,9cdcbf3c-225f-418c-9f86-3261cbff34c4,199,208,6.812642,6.812642,9.137864,7.588541


In [37]:
ride_life_cycle["Pickup_TO_Dropoff"] = ride_life_cycle['Dropoff_TS'] - ride_life_cycle['Pickup_TS']

In [38]:
ride_life_cycle.sort_values(by='Pickup_TO_Dropoff')

,Sim_ID,Ride_ID,Origin,Destination,Request_TS,Schedule_TS,Pickup_TS,Dropoff_TS,Pickup_TO_Dropoff
1506,4,9cdcbf3c-225f-418c-9f86-3261cbff34c4,199,208,6.812642,6.812642,9.137864,7.588541,-1.549323
1464,4,dfa39b3d-9955-429c-a8e5-8384b6fdb225,7,223,6.799523,6.799523,10.092021,9.199493,-0.892528
1498,4,beea018d-9196-4e03-a88a-ddf451af72c7,59,7,6.805370,6.805370,9.199493,8.362521,-0.836973
1446,4,4887cb86-831d-4af0-b0df-c6fb5a2a57d1,228,144,6.771949,6.771949,6.878502,6.898274,0.019772
1031,4,20ddd2c5-f1ee-42f0-9364-2fdab4c9ed10,44,139,6.175700,6.175700,11.592848,11.699559,0.106711
1445,4,4887cb86-831d-4af0-b0df-c6fb5a2a57d1,228,144,6.771949,6.771949,6.771949,6.898274,0.126325
1309,4,1e5e79a5-51e8-4923-8196-8ca18874217f,190,203,6.614239,6.614239,6.636448,6.771710,0.135263
976,4,bd523871-01b8-46ad-8efc-fea343f37579,180,182,6.106526,6.106526,6.297486,6.470465,0.172979
1360,4,ccb99ff5-5999-4677-bdce-f38b268244ab,149,135,6.667734,6.667734,6.959391,7.143681,0.184290
1507,4,9cdcbf3c-225f-418c-9f86-3261cbff34c4,199,208,6.812642,6.812642,9.137864,9.323528,0.185664


In [33]:
len(ride_life_cycle)

607

In [34]:
ride_life_cycle.dtypes

Sim_ID                 int64
Ride_ID               object
Origin                 int64
Destination            int64
Request_TS           float64
Schedule_TS          float64
Pickup_TS            float64
Dropoff_TS           float64
Pickup_TO_Dropoff    float64
dtype: object

## Closing the database.

In [94]:
c.close()
conn.close()